In [2]:
## to access the google drive with the google account
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import numpy as np
import imutils
import cv2
from matplotlib import pyplot as plt

def Detect_Feature_And_KeyPoints(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect and extract features from the image
    descriptors = cv2.xfeatures2d.SIFT_create()
    (Keypoints, features) = descriptors.detectAndCompute(image, None)

    Keypoints = np.float32([i.pt for i in Keypoints])
    return (Keypoints, features)
    
def get_Allpossible_Match(featuresA,featuresB):

    # compute the all matches using euclidean distance and opencv provide
    #DescriptorMatcher_create() function for that
    match_instance = cv2.DescriptorMatcher_create("BruteForce")
    All_Matches = match_instance.knnMatch(featuresA, featuresB, 2)

    return All_Matches

def All_validmatches(AllMatches, lowe_ratio):
    #to get all valid matches according to lowe concept..
    valid_matches = []

    for val in AllMatches:
        if len(val) == 2 and val[0].distance < val[1].distance * lowe_ratio:
            valid_matches.append((val[0].trainIdx, val[0].queryIdx))

    return valid_matches

def getwarp_perspective(imageA,imageB,Homography):
    ## given two images and H matrix to generate the panaroma
    val = imageA.shape[1] + imageB.shape[1]
    result_image = cv2.warpPerspective(imageA, Homography, (val , imageA.shape[0]))

    return result_image

def draw_Matches(imageA, imageB, KeypointsA, KeypointsB, matches, status):

    (hA,wA) = imageA.shape[:2]
    (hB, wB) = imageB.shape[:2]
    vis = np.zeros((max(hA, hB), wA + wB, 3), dtype="uint8")
    vis[0:hA, 0:wA] = imageA
    vis[0:hB, wA:] = imageB

    # loop over the matches
    for ((trainIdx, queryIdx), s) in zip(matches, status):
        if s == 1:
            ptA = (int(KeypointsA[queryIdx][0]), int(KeypointsA[queryIdx][1]))
            ptB = (int(KeypointsB[trainIdx][0]) + wA, int(KeypointsB[trainIdx][1]))
            cv2.line(vis, ptA, ptB, (0, 255, 0), 1)



In [4]:
# Step 1: read images and resize images
filename = ['q11.jpg', 'q22.jpg']

images = []
img_path = '/content/drive/My Drive/Computer Vision/Assignment3/'
for i in range(len(filename)):
    ## read image and append
    images.append(cv2.imread(img_path+filename[i]))
    ## resize image to 400x400
    images[i] = imutils.resize(images[i], width=400, height=400)

#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsB, featuresB) = Detect_Feature_And_KeyPoints(imageB)

#Step 3: get the valid matched points
AllMatches = get_Allpossible_Match(featuresA, featuresB);
valid_matches = All_validmatches(AllMatches, lowe_ratio=0.75)

#Step 4: Call cv2.findHomography
## the goal is to calculate the 3x3 H matrix

# construct the two sets of points
pointsA = np.float32([KeypointsA[i] for (_,i) in valid_matches])
pointsB = np.float32([KeypointsB[i] for (i,_) in valid_matches])

(Homograpgy, status) = cv2.findHomography(pointsA, pointsB, cv2.RANSAC, 4.0)

## Step 5: to get perspective of image using computed homography
result_image = getwarp_perspective(imageA, imageB, Homograpgy)
result_image[0:imageB.shape[0], 0:imageB.shape[1]] = imageB

draw_Matches(imageA, imageB, KeypointsA, KeypointsB, valid_matches, status)

## Save the images
cv2.imwrite(img_path+"image_output.jpg",result_image)



True

This code is given in https://github.com/allan-tulane/CMPS3660-6660/blob/main/Image_Stitching.ipynb
Most of them are implemented using existing libraries

The below one is simplified to panorama over 2 images.
There are 5 steps listed in the program including 
- Step 1: read two images and resize to the same size
- Step 2: detect the features and keypoints from SIFT
- Step 3: get the valid matched points out of all
- Step 4: Call cv2.findHomography to calculate the 3x3 H matrix
- Step 5: to get perspective of image using computed homography

## To Do
This assignment aims to reimplement Step 4 instead of using *cv2.findHomography*

This function aims to calcutlate the H matrix (3x3) given all matched pairs from image A and image B. 

- First, check the Concept *The direct linear transform (DLT)* from page 35-44 of slides https://github.com/allan-tulane/CMPS3660-6660/blob/main/slides/W5_1_homographies_slides.pdf

- Second, check *Estimating homography using RANSAC* in Page 69 to recalculate H. 
  - When we count inliers, we can use Squared SUM Error (SSE) to determine each line. That is, given a pair (p, q), we can calculate the SSE of the converted Hp, and q.
  - If their SSE is smaller than a threshold, that is an inlier. The threshold is manually set. You can try different values.

Note that you first use the given two images, then you use your own two images.




### 1. DLT (direct linear transformation)

In [5]:
# Step 1: read images and resize images
filename = ['q11.jpg', 'q22.jpg']

images = []
img_path = '/content/drive/My Drive/Computer Vision/Assignment3/'
for i in range(len(filename)):
    ## read image and append
    images.append(cv2.imread(img_path+filename[i]))
    ## resize image to 400x400
    images[i] = imutils.resize(images[i], width=400, height=400)

#Step 2: detect the features and keypoints from SIFT
(imageB, imageA) = images
(KeypointsA, featuresA) = Detect_Feature_And_KeyPoints(imageA)
(KeypointsPrime, featuresPrime) = Detect_Feature_And_KeyPoints(imageB)


#### TODO
1. Get matched feature points (SIFT)

2. Linear equations 
  - Ai = [-x, -y, -1, 0, 0, 0, xx', yx', x' |  0,  0,  0,-x,-y,-1, xy', yy', y' ]
    - i is for each h
  - h = [h1, h2, h3, h4, h5, h6, h7, h8, h9]
  - Where, A + h = 0



#### Solving DLT
For each {x, x'}, where x is the point from img1 and x' is the point of interest from SIFT(img2). 
  - Solve x' = Hx

1. For each correspondence, create 2x9 matrix: Ai
2. Concatenate into a single 2n x 9 matrix: A
3. Compute SVD of A = UΣVᵀ
  - Solution is the column of 9
corresponding to smallest singular
value

get eigenvalue plug into svd to get H


A =  UΣVᵀ
 - solve a*at for eigenvalues
 - take eigen vector with smallest eigen value
 min eig(a,aT)


In [6]:
#Step 3: Homographies
def get_Homographies(keypointA, keypointsPrime):
  H = []

  for keypointA, keypointsPrime in zip(KeypointsA, KeypointsPrime):

    # INFO: x = kp[0], y = kp[1]
    #[-x, -y, -1, 0, 0, 0, xx', yx', x' |  0,  0,  0,-x,-y,-1, xy', yy', y' ]
    x = keypointA[0]
    y = keypointA[1]

    x_prime = keypointsPrime[0]
    y_prime = keypointsPrime[1]

    # Steps 1/2
    n = 9 # number of concat matricies (0-8 = 9 total)
    A = np.matrix([[-x, -y, -1, 0, 0, 0, x*x_prime, y*x_prime, x_prime], [0, 0, 0, -x, -y, -1, x*y_prime, y*y_prime, y_prime]] * n)


    # Step 3: Compute SVD
    u, svd, v = np.linalg.svd(A, compute_uv=True)

    # Step 4: Get vector of the Smallest Singular value
    
    smallest_vector = []
    smallest_value = np.inf
    
    for vector in v.A: 
      for value in vector:
        if value < smallest_value:
          smallest_value = value
          smallest_vector = vector

    h = smallest_vector.reshape(3,3)

    h = h / h[2][2] #Normalization

    H.append(h)

  return H




In [7]:
get_Homographies(KeypointsA, KeypointsPrime)[0]

array([[-1.86217951e-04, -1.56931348e-02, -6.93327299e-05],
       [-1.26872177e-02, -1.06918918e+00, -4.72370914e-03],
       [ 2.68585924e+00,  2.26345252e+02,  1.00000000e+00]])

### RANSAC: Image Correspondences 

1. Randomly sample the number of points required to fit the model


2. Solve for model parameters using samples


3. Score by the fraction of inliers within a preset threshold of the model

Repeat 1-3 until the best model is found with high confidence

#### Randomly sample 
  RANSAC loop
1. Get four point correspondences (randomly)
2. Compute H using DLT
3. Count inliers
4. Keep H if largest number of inliers


In [9]:
## RANSAC ##
import random
# 1. 4 random correspondences
def generate_randoms(keypointsA, keypointsPrime, num_randoms):
  return [[random.choice(keypointsA) for i in range(num_randoms)], [random.choice(keypointsPrime) for i in range(num_randoms)]]

random_keypoints = generate_randoms(KeypointsA, KeypointsPrime, 4)

# 2. Homographies of the 4 correspondences
H = []
for keypoint1, keypoint2 in zip(random_keypoints[0], random_keypoints[1]):
  H.append(get_Homographies(keypoint1,keypoint2))
H

# 3. Count inliners...??


[[array([[-1.86217951e-04, -1.56931348e-02, -6.93327299e-05],
         [-1.26872177e-02, -1.06918918e+00, -4.72370914e-03],
         [ 2.68585924e+00,  2.26345252e+02,  1.00000000e+00]]),
  array([[ 3.79933472e-01, -6.67621136e-03, -1.88378626e+00],
         [ 8.55640399e-02,  3.90036397e+00, -1.24158472e+03],
         [ 1.56999896e-01, -7.53577693e-03,  1.00000000e+00]]),
  array([[-3.98886562e-04, -2.67382654e-02, -9.42997847e-05],
         [-1.91610956e-02, -1.28441143e+00, -4.52982718e-03],
         [ 4.22998393e+00,  2.83545354e+02,  1.00000000e+00]]),
  array([[-4.63989371e-04, -2.98417012e-02, -1.07065183e-04],
         [-1.91449065e-02, -1.23131394e+00, -4.41767212e-03],
         [ 4.33370911e+00,  2.78724609e+02,  1.00000000e+00]]),
  array([[-6.90604278e-04, -4.44165487e-02, -1.59356394e-04],
         [-2.12147692e-02, -1.36443816e+00, -4.89529131e-03],
         [ 4.33370933e+00,  2.78724611e+02,  1.00000000e+00]]),
  array([[-7.19725723e-04, -4.36282977e-02, -1.59356398e-04]